In [ ]:
!pip install -q kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d hazemfahmy/openned-closed-eyes

In [ ]:
!unzip /content/openned-closed-eyes.zip

In [ ]:
import os
import random
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.layers import Dense, Flatten
from keras.models import Sequential
from keras.optimizers import Adam

In [ ]:
tpath = '/content/TrainingSet/TrainingSet'
vpath = '/content/ImprovementSet/ImprovementSet/ImprovementSet'

In [ ]:
training_data = keras.preprocessing.image_dataset_from_directory(
    vpath,
    batch_size =32,
    image_size =(227,227),

    shuffle = True,
    seed =123,
    subset ='training',
    validation_split=0.01
    )
validation_data =keras.preprocessing.image_dataset_from_directory(
    tpath,
    batch_size = 32,
    image_size =(227,227),

    shuffle = True,
    seed =123,
    validation_split =0.99,
    subset ='validation'

    )

In [ ]:
resnet_model = Sequential()

pretrained_model= keras.applications.ResNet50(include_top=False,
                   input_shape=(227,227,3),
                   pooling='avg',classes=2,
                   weights='imagenet')
for layer in pretrained_model.layers:
        layer.trainable=False

resnet_model.add(pretrained_model)
resnet_model.add(Flatten())
resnet_model.add(Dense(512, activation='relu'))
resnet_model.add(Dense(2, activation='softmax'))

In [ ]:
resnet_model.compile(loss='sparse_categorical_crossentropy',
                     optimizer=keras.optimizers.SGD(lr=0.0001),
                     metrics=['accuracy'])

In [ ]:
epochs=2
history = resnet_model.fit(

  training_data,
  validation_data=validation_data,
  epochs=epochs
)

In [ ]:
testdata = '/content/TestSet/TestSet'

In [ ]:
testdata2 = keras.preprocessing.image_dataset_from_directory(
    testdata,
    batch_size =32,
    image_size =(227,227),

    shuffle = True,
    seed =123,
    subset ='training',
    validation_split=0.01
    )

In [ ]:
print("evaluating network...")
predIdxs = resnet_model.predict(testdata2)

predIdxs = np.argmax(predIdxs, axis=1)



In [ ]:
resnet_model.save("Drowsiness Detection .model", save_format="h5")

N = 2
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), history.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), history.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), history.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), history.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot.png")


In [ ]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import imutils
import time
import cv2
import os

In [ ]:
prototxtPath = r"/content/deploy.prototxt"
weightsPath = r"/content/res10_300x300_ssd_iter_140000.caffemodel"
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

# load the face mask detector model from disk
maskNet = load_model("/content/Drowsiness Detection .model")

In [ ]:
def detect_and_predict_mask(frame, faceNet, maskNet):

	(h, w) = frame.shape[:2]
	blob = cv2.dnn.blobFromImage(frame, 1.0, (227, 227),
		(104.0, 177.0, 123.0))

	faceNet.setInput(blob)
	detections = faceNet.forward()
	print(detections.shape)

	faces = []
	locs = []
	preds = []

	for i in range(0, detections.shape[2]):
		confidence = detections[0, 0, i, 2]

		if confidence > 0.5:
			box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
			(startX, startY, endX, endY) = box.astype("int")

			(startX, startY) = (max(0, startX), max(0, startY))
			(endX, endY) = (min(w - 1, endX), min(h - 1, endY))

			face = frame[startY:endY, startX:endX]
			face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
			face = cv2.resize(face, (227, 227))
			face = img_to_array(face)
			face = preprocess_input(face)

			faces.append(face)
			locs.append((startX, startY, endX, endY))

	if len(faces) > 0:
		faces = np.array(faces, dtype="float32")
		preds = maskNet.predict(faces, batch_size=32)

	return (locs, preds)

In [ ]:
print(" starting video stream...")
vs = VideoStream(src=0).start()


while True:
	frame = vs.read()
    frame = imutils.resize(frame, width=400)

	(locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)

	for (box, pred) in zip(locs, preds):
		(startX, startY, endX, endY) = box
		(Closed, Opened) = pred

		label = "Closed" if Closed > Opened else "Opened"
		color = (0, 255, 0) if label == "Closed" else (0, 0, 255)

		label = "{}: {:.2f}%".format(label, max(Closed, Opened) * 100)

		cv2.putText(frame, label, (startX, startY - 10),
			cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
		cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

	cv2.imshow("Frame", frame)
	key = cv2.waitKey(1) & 0xFF

	if key == ord("q"):
		break

cv2.destroyAllWindows()
vs.stop()


In [ ]:
from google.colab.patches import cv2_imshow
import cv2
img = cv2.imread('/content/fd5c4002-93a2-44fd-8d4b-51209e7dcdaa.jpg', cv2.IMREAD_UNCHANGED)
frame = imutils.resize(img, width=400)
(locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)
for (box, pred) in zip(locs, preds):
		(startX, startY, endX, endY) = box
		(Closed, Opened) = pred


		label = "Closed" if Closed > Opened else "Opened"
		color = (0, 255, 0) if label == "Closed"else (0, 0, 255)

		label = "{}: {:.2f}%".format(label, max(Closed, Opened) * 100)

		cv2.putText(frame, label, (startX, startY - 10),
			cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
		cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

cv2_imshow(frame)

In [ ]:
from google.colab.patches import cv2_imshow
import cv2
img = cv2.imread('/content/2014_8_25_10_11_23_418.jpg', cv2.IMREAD_UNCHANGED)
frame = imutils.resize(img, width=400)
(locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)
for (box, pred) in zip(locs, preds):
		(startX, startY, endX, endY) = box
		(Closed, Opened) = pred


        if Closed > Opened:
            engine.say("Alert!!!! WAKE UP DUDE")
            engine.runAndWait()
        else:
            engine.say("Alert!!!! good man")
            engine.runAndWait()


		label = "Closed" if Closed > Opened else "Opened"
		color = (0, 255, 0) if label == "Closed"else (0, 0, 255)



		label = "{}: {:.2f}%".format(label, max(Closed, Opened) * 100)

		cv2.putText(frame, label, (startX, startY - 10),
			cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
		cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

cv2_imshow(frame)